In [1]:
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

In [134]:
import qutip as qt
import numpy as np
from opentn.circuits import get_unitary_adchannel, quantum_circuit, get_unitary_beamsplitter
from opentn.channels import get_krauss_from_unitary, test_trace_preserving, quantum_channel
from opentn.states.qubits import convert_to_comp_basis
from opentn.states.qubits import up, down, plus

In [ ]:
# Ideas: 
# create kruass operators of damping channel using qutip and  convert to stinespring to see what we get (DONE)
# start with UAE (both from circuits and beam splitter and obtain krauss operators) (DONE->wrong)
# see how the krauss operators are obtained in qutip (?)
# use both create and anhiliation and sigmap and siman (DONE)


In [13]:
X = qt.sigmax()
I = qt.qeye(2)
qt.spre(X)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True
Qobj data =
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

In [15]:
qt.tensor(I,X)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

In [20]:
qt.spost(X)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True
Qobj data =
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

In [19]:
qt.spost(X.dag())

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True
Qobj data =
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

In [17]:
qt.tensor(X,I)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

In [18]:
qt.to_super(X)

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True
Qobj data =
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

In [ ]:
# check the superoperator form of the kruass operators by hand by just taking the sum of the tensor product of the individual elements
# it seems like i need to create a krauss operator and then use to_stinespring

In [140]:
gamma = 0.5

In [45]:
#checking if I get the superoperator from the channel correctly:
U = get_unitary_adchannel(gamma=gamma)
E0, E1 = get_krauss_from_unitary(U)
super_channel = np.kron(E1.conj(), E1) + np.kron(E0.conj(), E0)
qt.Qobj(super_channel)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.         0.         0.         0.5       ]
 [0.         0.70710678 0.         0.        ]
 [0.         0.         0.70710678 0.        ]
 [0.         0.         0.         0.5       ]]

In [37]:
ap = qt.create(2)
am = qt.destroy(2)
M0 = (I - gamma*ap*am).sqrtm()
M1 = np.sqrt(gamma)*am

In [108]:
qt_super_channel = qt.kraus_to_super([M0,M1])
qt.Qobj(convert_to_comp_basis(super_channel, env_first=True, num_levels=2))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.         0.         0.         0.5       ]
 [0.         0.70710678 0.         0.        ]
 [0.         0.         0.70710678 0.        ]
 [0.         0.         0.         0.5       ]]

In [91]:
qt.to_kraus(qt_super_channel)

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[1.         0.        ]
  [0.         0.70710678]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
 Qobj data =
 [[0.         0.70710678]
  [0.         0.        ]]]

In [46]:
Uevol, Uevol_dagger = qt.to_stinespring(qt_super_channel)
Uevol

Quantum object: dims = [[2, 2], [2]], shape = (4, 2), type = oper, isherm = False
Qobj data =
[[-1.          0.        ]
 [ 0.          0.70710678]
 [ 0.         -0.70710678]
 [ 0.          0.        ]]

In [49]:
Uevol_dagger

Quantum object: dims = [[2, 2], [2]], shape = (4, 2), type = oper, isherm = False
Qobj data =
[[-1.          0.        ]
 [ 0.          0.70710678]
 [ 0.         -0.70710678]
 [ 0.          0.        ]]

In [158]:
qt.Qobj(convert_to_comp_basis(U, env_first=True, num_levels=2))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678  0.         -0.70710678]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.          0.          1.          0.        ]]

In [114]:
U_bs = get_unitary_beamsplitter(gamma=gamma)
qt.Qobj(convert_to_comp_basis(U_bs, env_first=True, num_levels=2))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678  0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]

In [86]:
# try to create unitary of beam splitter from qutip functions
# here im comparing what happens if i use the different operators, it is so weird, which one should I use?
theta = np.arcsin(np.sqrt(gamma))
sigmap = qt.sigmap()
sigmam = qt.sigmam()
#U_bs_qt = (theta*(qt.tensor(sigmap,sigmam) - qt.tensor(sigmam,sigmap))).expm() # WRONG.
U_bs_qt = (theta*(qt.tensor(ap,am) - qt.tensor(am,ap))).expm() # RIGHT
U_bs_qt

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678  0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]

In [116]:
print(sigmam*qt.basis(2,0))
ap == sigmam # the ap is the creation operator for the harmonic oscillator and the sigma minus is the spin operator minus

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]


True

In [68]:
qt.to_kraus(qt.to_super(U_bs_qt))

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
 Qobj data =
 [[ 1.          0.          0.          0.        ]
  [ 0.          0.70710678 -0.70710678  0.        ]
  [ 0.          0.70710678  0.70710678  0.        ]
  [ 0.          0.          0.          1.        ]]]

In [144]:
k_list_bs = get_krauss_from_unitary(U_bs.data, env_init=qt.basis(2,1))
test_trace_preserving(k_list_bs)
k_list_bs

Trace Preserving


[array([[ 0.        +0.j,  0.        +0.j],
        [-0.70710678+0.j,  0.        +0.j]]),
 array([[0.70710678+0.j, 0.        +0.j],
        [0.        +0.j, 1.        +0.j]])]

In [142]:
k_list_qt = get_krauss_from_unitary(U_bs_qt.data, env_init=qt.basis(2,1))
k_list_qt

[array([[0.        +0.j, 0.        +0.j],
        [0.70710678+0.j, 0.        +0.j]]),
 array([[0.70710678+0.j, 0.        +0.j],
        [0.        +0.j, 1.        +0.j]])]

In [153]:
k_list = (get_krauss_from_unitary(U, env_init=qt.basis(2,1)))
k_list

[array([[ 0.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -0.70710678+0.j]]),
 array([[0.        +0.j, 0.70710678+0.j],
        [1.        +0.j, 0.        +0.j]])]

In [137]:
quantum_channel(state=plus, krauss_list=k_list)

array([[0.25      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.75      +0.j]])

In [139]:
quantum_channel(state=plus, krauss_list=k_list_bs)

array([[0.25      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.75      +0.j]])

In [145]:
quantum_channel(state=plus, krauss_list=k_list_qt)

array([[0.25      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.75      +0.j]])

In [150]:
quantum_circuit(initial_state=[plus,down], U=U_bs_qt.data) # this is the only one that produces different results for the environment, the other two (U, U_bs) have a minus on off diagonal

(array([[0.25      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.75      +0.j]]),
 array([[0.25      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.75      +0.j]]))